In [ ]:
#| default_exp derivations

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Dict, Any

In [ ]:
#| export
def normalize_mass_unit(unit_text: Optional[str]) -> str:
    """Normalize mass/volume units to canonical form for comparison."""
    if not unit_text:
        return ''
    u = unit_text.lower().strip()
    if u in ('mcg', 'microgram', 'micrograms'):
        return 'mcg'
    if u in ('mg', 'milligram', 'milligrams'):
        return 'mg'
    if u in ('g', 'gram', 'grams'):
        return 'g'
    if u in ('ml', 'mls', 'milliliter', 'milliliters', 'millilitre', 'millilitres', 'mL'):
        return 'ml'
    if u in ('unit', 'units'):
        return 'unit'
    return u


In [ ]:
#| export
def compute_dosage_per_administration(dosage_field: Dict[str, Any]) -> Dict[str, Any]:
    """Compute dosage per administration fields."""
    result = {}
    
    strength = dosage_field.get('strength')
    dosage = dosage_field.get('dosage_max') or dosage_field.get('dosage')
    dosage_unit = dosage_field.get('dosage_unit')
    strength_unit = dosage_field.get('strength_unit')
    
    mass_volume_units = ['mg', 'mcg', 'g', 'ml', 'mL']
    
    # Simple calculation
    if strength and dosage:
        result['dosage_per_administration'] = strength * dosage
        result['dosage_per_administration_unit'] = strength_unit
    elif dosage and dosage_unit in mass_volume_units:
        result['dosage_per_administration'] = dosage
        result['dosage_per_administration_unit'] = dosage_unit
    
    # Compound numerator/denominator
    if dosage_field.get('strength_numerator') is not None and dosage:
        result['dosage_numerator_per_administration'] = dosage
        result['dosage_numerator_per_administration_unit'] = dosage_unit
        result['dosage_denominator_per_administration'] = 1.0
        result['dosage_denominator_per_administration_unit'] = 'administration'
    
    return result


In [ ]:
#| export
def compute_dosage_per_period(dosage_field: Dict[str, Any], per_admin: Dict[str, Any]) -> Dict[str, Any]:
    """Compute dosage per period fields."""
    result = {}
    
    freq = dosage_field.get('frequency_max') or dosage_field.get('frequency')
    period = dosage_field.get('period_max') or dosage_field.get('period')
    dosage = dosage_field.get('dosage_max') or dosage_field.get('dosage')
    
    # Per period
    if per_admin.get('dosage_per_administration') and freq and period:
        result['dosage_per_period'] = per_admin['dosage_per_administration'] * freq / period
        result['dosage_per_period_unit'] = per_admin.get('dosage_per_administration_unit')
    
    # Compound
    if dosage_field.get('strength_numerator') is not None:
        if dosage and freq:
            result['dosage_numerator_per_period'] = dosage * freq
            result['dosage_numerator_per_period_unit'] = dosage_field.get('dosage_unit')
        if period:
            result['dosage_denominator_per_period'] = period
            result['dosage_denominator_per_period_unit'] = dosage_field.get('period_unit')
    
    return result


In [ ]:
#| export
def compute_dosage_per_duration(dosage_field: Dict[str, Any], per_period: Dict[str, Any]) -> Dict[str, Any]:
    """Compute dosage per duration (total course) fields."""
    result = {}
    
    duration = dosage_field.get('duration_max') or dosage_field.get('duration')
    duration_unit = dosage_field.get('duration_unit')
    period_unit = dosage_field.get('period_unit')
    
    conversion = {'day': 1, 'week': 7, 'month': 30, 'hour': 1/24, 'minute': 1/1440}
    
    # Per duration
    if per_period.get('dosage_per_period') and duration and duration_unit and period_unit:
        if duration_unit in conversion and period_unit in conversion:
            duration_in_period_units = duration * conversion[duration_unit] / conversion[period_unit]
            result['dosage_per_duration'] = per_period['dosage_per_period'] * duration_in_period_units
            result['dosage_per_duration_unit'] = per_period.get('dosage_per_period_unit')
    
    # Compound numerator
    if per_period.get('dosage_numerator_per_period') and duration and duration_unit and period_unit:
        if duration_unit in conversion and period_unit in conversion:
            duration_in_period_units = duration * conversion[duration_unit] / conversion[period_unit]
            result['dosage_numerator_per_duration'] = per_period['dosage_numerator_per_period'] * duration_in_period_units
            result['dosage_numerator_per_duration_unit'] = per_period.get('dosage_numerator_per_period_unit')
    
    # Compound denominator
    if dosage_field.get('strength_numerator') is not None and duration:
        result['dosage_denominator_per_duration'] = duration
        result['dosage_denominator_per_duration_unit'] = duration_unit
    
    return result


In [ ]:
#| export
def derive_all_fields(dosage_field: Dict[str, Any]) -> Dict[str, Any]:
    """Compute all derived dosage fields."""
    per_admin = compute_dosage_per_administration(dosage_field)
    per_period = compute_dosage_per_period(dosage_field, per_admin)
    per_duration = compute_dosage_per_duration(dosage_field, per_period)
    
    return {**per_admin, **per_period, **per_duration}
